In [1]:
import data_processing as dclean
from config import app
from models import Hive

In [2]:
with app.app_context():
    hives = [hive.to_dict() for hive in Hive.query.all()]

In [3]:
graph_data = dclean.process_data_for_graphing(hives)

In [4]:
graph_data['aggregated']['weight']

[159.8168672803875,
 169.76025722785238,
 148.88756840466144,
 165.54666988891745,
 193.62622109663369,
 45.274237084661074,
 123.1380908447252,
 133.29416495751875,
 85.11734240816776,
 133.83730446904985,
 173.2046667465922,
 148.25133726014695,
 88.6155100830513,
 30.132706930104057,
 90.71866397413446,
 150.6228585572417,
 130.05775126770175,
 180.1177495186825,
 37.09979929612215,
 191.45204302153985,
 102.32811062152223,
 179.3997227735853,
 2.999779343966602,
 64.04648000900353,
 163.67390401792056,
 157.55777475013738,
 132.60981561392603]

In [13]:

df_subset = df_aggregated.head(10)

df_subset.to_dict()

{'date_added': {0: '2022-10-12',
  1: '2022-10-12',
  2: '2022-10-12',
  3: '2022-10-12',
  4: '2022-10-12',
  5: '2022-10-12',
  6: '2023-02-01',
  7: '2023-02-01',
  8: '2023-02-01',
  9: '2023-02-01'},
 'material': {0: 'Polystyrene',
  1: 'Polystyrene',
  2: 'Polystyrene',
  3: 'Polystyrene',
  4: 'Polystyrene',
  5: 'Polystyrene',
  6: 'Polystyrene',
  7: 'Polystyrene',
  8: 'Polystyrene',
  9: 'Polystyrene'},
 'city': {0: 'Amandastad',
  1: 'Amandastad',
  2: 'Amandastad',
  3: 'Amandastad',
  4: 'Amandastad',
  5: 'Amandastad',
  6: 'Sanchezview',
  7: 'Sanchezview',
  8: 'Sanchezview',
  9: 'Sanchezview'},
 'state': {0: 'Michigan',
  1: 'Michigan',
  2: 'Michigan',
  3: 'Michigan',
  4: 'Michigan',
  5: 'Michigan',
  6: 'Michigan',
  7: 'Michigan',
  8: 'Michigan',
  9: 'Michigan'},
 'honey_pull_id': {0: 139,
  1: 140,
  2: 141,
  3: 142,
  4: 143,
  5: 144,
  6: 146,
  7: 147,
  8: 148,
  9: 149},
 'date_reset': {0: '2022-10-12',
  1: '2023-01-30',
  2: '2023-05-31',
  3: '2023

In [18]:
json_normalized = df_normalized.head().to_dict()

In [19]:
json_aggregated = df_aggregated.head().to_dict()

In [20]:
graph_data = {'normalized': json_normalized, 'aggregated': json_aggregated}

In [21]:
graph_data

{'normalized': {'humidity': {0: 86.0, 1: 85.0, 2: 58.7, 3: 50.7, 4: 87.8},
  'varroa_mite_count': {0: 31.0, 1: 58.0, 2: 60.0, 3: 17.0, 4: 91.0},
  'bias': {0: 2, 1: 2, 2: 2, 3: 3, 4: 2},
  'inspection_id': {0: 2121, 1: 2122, 2: 2123, 3: 2124, 4: 2125},
  'has_chalkbrood': {0: False, 1: True, 2: True, 3: True, 4: False},
  'ants_present': {0: True, 1: False, 2: True, 3: False, 4: True},
  'apistan_dosage': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0},
  'stability_in_hive': {0: 'High', 1: 'Low', 2: 'Low', 3: 'High', 4: 'Medium'},
  'num_pollen_patties': {0: 2, 1: 0, 2: 2, 3: 0, 4: 0},
  'mice_present': {0: True, 1: True, 2: False, 3: True, 4: False},
  'honey_pull_id': {0: 122, 1: 122, 2: 122, 3: 122, 4: 122},
  'robber_bees_present': {0: True, 1: False, 2: True, 3: False, 4: False},
  'hive_beetles_present': {0: True, 1: True, 2: True, 3: False, 4: False},
  'wasps_hornets_present': {0: True, 1: False, 2: True, 3: False, 4: False},
  'num_sugar_syrup_frames': {0: 1, 1: 2, 2: 2, 3: 1, 4: 1}

In [3]:
death_count = df_normalized[['fate', 'date_checked']].loc[df_normalized['fate']=='Dead']
len(death_count)

240